Hamilton,W.L. *Graph Representation Learning*. 2020

# 이웃 중복 감지

**Neighborhood Overlap Detection**

[전 포스트]()에서 소개한 방법은 분류 작업에 유용한 피쳐들이기는 하지만, 노드 간의 관계(*relationships*)를 정량화하지는 못한다. 따라서 관계예측(relation prediction)에 유용하지 못하다.

<img src="img/fig2_3.png" width='500'>

이 포스트에서는 노

## Local Overlap Measures

## Global Overlap Measures

# 참고자료

[1] Hamilton, W. L. (2020). *Graph Representation Learning.* Morgan & Claypool Publishers.